# AG2: Tools (Interoperability) (LangChain Tools Integration)

- LangChain Tools Integration<br>
  - https://docs.ag2.ai/docs/user-guide/basic-concepts/tools/interop/langchain

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## LangChain Tools Integration

### Introduction
- LangChain Community Tools
  - https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools

### Installation

```python
pip install ag2[openai,interop-langchain]

# If you have been using autogen or pyautogen, all you need to do is upgrade it using:
pip install -U autogen[openai,interop-langchain]
pip install -U pyautogen[openai,interop-langchain]
```
as pyautogen, autogen, and ag2 are aliases for the same PyPI package

**Wikipedia Tool**
- https://python.langchain.com/docs/integrations/tools/wikipedia/

```python
pip install wikipedia
```

### Imports

In [2]:
import os

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# from autogen import AssistantAgent, UserProxyAgent, LLMConfig
from autogen import AssistantAgent, UserProxyAgent
from autogen.interop import Interoperability

### Agent Configuration

In [3]:
# config_list = [{"api_type": "openai", "model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]
config_list = {"api_type": "openai", "model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding_work",
        "use_docker": False,  # Set to True if you want to use Docker
    }
)

chatbot = AssistantAgent(
    name="chatbot",
    # llm_config=LLMConfig(config_list=config_list),
    llm_config=config_list,
)

### Tool Integration

In [4]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
langchain_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

interop = Interoperability()
ag2_tool = interop.convert_tool(tool=langchain_tool, type="langchain")

ag2_tool.register_for_execution(user_proxy)
ag2_tool.register_for_llm(chatbot)

### Initiating the Chat

In [5]:
message = "Tell me about the history of the United States"

In [6]:
chat_result = user_proxy.initiate_chat(recipient=chatbot, message=message, max_turns=2)

User (to chatbot):

Tell me about the history of the United States

--------------------------------------------------------------------------------
chatbot (to User):

***** Suggested tool call (call_LGSURIWPSRNl1YcjTg6FBV2M): wikipedia *****
Arguments: 
{"query":"History of the United States"}
**************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION wikipedia...
Call ID: call_LGSURIWPSRNl1YcjTg6FBV2M
Input arguments: {'query': 'History of the United States'}
User (to chatbot):

***** Response from calling tool (call_LGSURIWPSRNl1YcjTg6FBV2M) *****
Error: 'tool_input'
**********************************************************************

--------------------------------------------------------------------------------
chatbot (to User):

***** Suggested tool call (call_5N5zTslFjxGY7VDebL1ABeuw): wikipedia *****
Arguments: 
{"query":"History of the United Sta

### Output

In [7]:
print(len(chat_result.chat_history))
# print(chat_result.chat_history[0])
# print(chat_result.chat_history[-1])

print(chat_result.chat_history)

4
[{'content': 'Tell me about the history of the United States', 'role': 'assistant', 'name': 'User'}, {'tool_calls': [{'id': 'call_LGSURIWPSRNl1YcjTg6FBV2M', 'function': {'arguments': '{"query":"History of the United States"}', 'name': 'wikipedia'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': "Error: 'tool_input'", 'tool_responses': [{'tool_call_id': 'call_LGSURIWPSRNl1YcjTg6FBV2M', 'role': 'tool', 'content': "Error: 'tool_input'"}], 'role': 'tool', 'name': 'User'}, {'tool_calls': [{'id': 'call_5N5zTslFjxGY7VDebL1ABeuw', 'function': {'arguments': '{"query":"History of the United States"}', 'name': 'wikipedia'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}]
